In [25]:
from tokenize import String
from textblob import TextBlob as tb
import pandas as pd
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer as Sia
from gensim.parsing.preprocessing import remove_stopwords
from pathlib import Path
from collections import defaultdict
import numpy as np
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS 
import matplotlib.pyplot as plt
import hvplot.pandas 
import hvplot.dask  
import tweepy
client = tweepy.Client("AAAAAAAAAAAAAAAAAAAAAGvbawEAAAAAxvxnJu9E%2F%2FPCe5Oz0irJC5BDvZM%3DEsqky8zPsYhadIeHKzCVsaB0UXGz28FFb2UBqtnNDOJkmfAkfN")
import GetOldTweets3 as got

%matplotlib inline

nltk.download('vader_lexicon')
nltk.download('words')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gunin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\gunin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [13]:
def tweets(start_time, end_time, iterations, query):
    datelist = pd.date_range(start_time, end_time, periods=iterations).tolist()
    tweets=[]
    date=[]
    #prob=[]
    #sentim=[]
    a=0
    for time in datelist:
        if a>1:
            #Fetching Tweets
            response = client.search_recent_tweets(query, max_results=100, start_time=start, end_time=time, tweet_fields=["created_at"])
            response.data
            if type(response.data) != type(None):
                #Data Cleaning
                for i in response.data:
                    tw=i
                    tw=str(i)
                    tw=tw.lower()
                    tw=tw.split()
                    tw=' '.join(word for word in tw)
                    #Sentiment analysis
                    if len(tw) > 0:                        
                        tweets.append(tw)
                        date.append(i.created_at)                        
        start=time
        a=a+1
        
    tweet_df=pd.DataFrame({'time':date, 'tweet':tweets})
    tweet_df=tweet_df.sort_values(by='time', ascending=True).set_index('time')
    return tweet_df

In [14]:
btc_tweets = tweets('2022-05-27', '2022-06-02', 100, 'btc OR bitcoin OR crypto -is:retweet lang:en')

In [15]:
eth_tweets = tweets('2022-05-27', '2022-06-02', 100, 'eth OR ethereum OR crypto -is:retweet lang:en')
eth_tweets.tail()

,tweet
time,
2022-06-01 23:59:59+00:00,@viamirror #blockflix is the first-ever tv str...
2022-06-01 23:59:59+00:00,rt @egor_shirinsky: night comes to the kingdom...
2022-06-01 23:59:59+00:00,rt @goshfinder: 💥 weekly most voted influencer...
2022-06-01 23:59:59+00:00,guys ! i made 3k to 33k after joined https://t...
2022-06-01 23:59:59+00:00,rt @_jokernft_: #nft 🎁giveaway 🎁 $eth 1️⃣ must...


In [16]:
bin_tweets = tweets('2022-05-27', '2022-06-02', 100, 'binance OR crypto -is:retweet lang:en')

In [17]:
xrp_tweets = tweets('2022-05-27', '2022-06-02', 100, 'xrp OR ripple OR crypto -is:retweet lang:en')

In [20]:
ada_tweets = tweets('2022-05-27', '2022-06-02', 100, 'ada OR cardano OR crypto -is:retweet lang:en')

In [21]:
sol_tweets = tweets('2022-05-27', '2022-06-02', 100, 'sol OR solana OR crypto -is:retweet lang:en')

In [22]:
matic_tweets = tweets('2022-05-27', '2022-06-02', 100, 'matic OR polygon OR crypto -is:retweet lang:en')

In [23]:
sand_tweets = tweets('2022-05-27', '2022-06-02', 100, 'sandbox crypto -is:retweet lang:en')

In [26]:
mana_tweets = tweets('2022-05-27', '2022-06-02', 100, 'mana OR decentraland OR crypto -is:retweet lang:en')

In [27]:
algo_tweets = tweets('2022-05-27', '2022-06-02', 100, 'algo OR algorand OR crypto -is:retweet lang:en')

In [28]:
sia = Sia()
from itertools import chain


def clean_text(input_str: str) -> str:
    input_str = input_str.lower()
    input_str = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", input_str)
    input_str = " ".join(input_str.split())
    input_str = re.sub("[^A-Za-z0-9]"," ",input_str)
    input_str = re.sub(r'^https?:\/\/.*[\r\n]*', '', input_str, flags=re.MULTILINE)
    input_str = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', input_str)
    input_str = re.sub(r"www.\S+",'',input_str)
    input_str = remove_stopwords(input_str)
    input_str = ''.join([i for i in input_str if not i.isdigit()])
    return input_str

def build_sentiment_df(tweet_dataframe, target_column):
    # getting the corresponding data in lists
    raw_tweets = tweet_dataframe[target_column].tolist()
    cleaned_tweets = [clean_text(i) for i in raw_tweets]
    nltk_polarity = [sia.polarity_scores(i)['compound'] for i in cleaned_tweets]
    
    # building the dataframe
    final_df = tweet_dataframe.copy()
    final_df['cleaned_tweets'] = cleaned_tweets
    final_df['nltk_polarity'] = nltk_polarity

    final_df = final_df[final_df['tweet'].apply(lambda x: len(x.split(',')) < 280)]
    final_df = final_df.drop_duplicates()
    final_df = final_df[final_df['tweet'].str.contains("rt ") == False]
    final_df = final_df[final_df['nltk_polarity'] != 0.0000] 

    return final_df



In [29]:
btc_data = build_sentiment_df(btc_tweets, "tweet")
btc_data.head()

,tweet,cleaned_tweets,nltk_polarity
time,,,
2022-05-27 02:54:09+00:00,@secgov is corrupt! @garygensler @uscongress #...,corrupt usa crypto xrp want truth justice,0.7184
2022-05-27 02:54:09+00:00,@twobitidiot why no one say something like lin...,like link oracle key infrastructure base crypt...,0.3612
2022-05-27 02:54:09+00:00,getting paid crypto to create whatever content...,getting paid crypto create content want post s...,0.3400
2022-05-27 02:54:11+00:00,institutional investors seek safe haven in cry...,institutional investors seek safe haven crypto...,0.1280
2022-05-27 02:54:12+00:00,really very excellent project. the project was...,excellent project project executed professiona...,0.5859


In [30]:
eth_data = build_sentiment_df(eth_tweets, "tweet")
eth_data.head()

,tweet,cleaned_tweets,nltk_polarity
time,,,
2022-05-27 02:54:22+00:00,@chitaglorya__ @twinutoken don't miss out on t...,t miss profitable crypto community join channe...,0.5423
2022-05-27 02:54:22+00:00,@justnthephotog i saw a cool bird was bout to ...,saw cool bird bout snag trying eth balance dol...,0.7650
2022-05-27 02:54:22+00:00,@buckleupbrandon are you gonna try to play the...,gonna try play bounce eth think bounce,0.3400
2022-05-27 02:54:22+00:00,🐳🐳🐳 nft whale trade ➡️transfer: [0x9cce3c] -&g...,nft whale trade transfer xccec gt xaee nft mba...,0.2732
2022-05-27 02:54:23+00:00,i believe this is a faithful project. the proj...,believe faithful project projector lot attract...,0.9118


In [31]:
eth_data.shape

(2364, 3)

In [32]:
btc_data.shape

(3094, 3)

In [33]:
bin_data = build_sentiment_df(bin_tweets, "tweet")
ada_data = build_sentiment_df(ada_tweets, "tweet")
xrp_data = build_sentiment_df(xrp_tweets, "tweet")
sol_data = build_sentiment_df(sol_tweets, "tweet")
matic_data = build_sentiment_df(matic_tweets, "tweet")
sand_data = build_sentiment_df(sand_tweets, "tweet")
algo_data = build_sentiment_df(algo_tweets, "tweet")
mana_data = build_sentiment_df(mana_tweets, "tweet")

In [34]:
bin_data.shape

(4017, 3)

In [35]:
ada_data.shape

(2400, 3)

In [36]:
xrp_data.shape

(5165, 3)

In [37]:
sol_data.shape

(1205, 3)

In [38]:
btc_data.shape

(3094, 3)

In [39]:
btc_data.head()

,tweet,cleaned_tweets,nltk_polarity
time,,,
2022-05-27 02:54:09+00:00,@secgov is corrupt! @garygensler @uscongress #...,corrupt usa crypto xrp want truth justice,0.7184
2022-05-27 02:54:09+00:00,@twobitidiot why no one say something like lin...,like link oracle key infrastructure base crypt...,0.3612
2022-05-27 02:54:09+00:00,getting paid crypto to create whatever content...,getting paid crypto create content want post s...,0.3400
2022-05-27 02:54:11+00:00,institutional investors seek safe haven in cry...,institutional investors seek safe haven crypto...,0.1280
2022-05-27 02:54:12+00:00,really very excellent project. the project was...,excellent project project executed professiona...,0.5859


In [40]:
ada_data

,tweet,cleaned_tweets,nltk_polarity
time,,,
2022-05-27 02:54:23+00:00,@dinoxdj99 ada ka jaehyun collector yg drop ? ...,ada ka jaehyun collector yg drop wkwkwkwk nnti...,0.0258
2022-05-27 02:54:23+00:00,@gyuacarat kpop merch nanti ada benefit pcnyaa...,kpop merch nanti ada benefit pcnyaaa dan keban...,0.4588
2022-05-27 02:54:24+00:00,@emfelwe beep 🚨 lagi cari app prem? ada dong! ...,beep lagi cari app prem ada dong ada netflix y...,0.4767
2022-05-27 02:54:24+00:00,@kelvisman @mononokeinuscam welcome to crypto 🤩,welcome crypto,0.4588
2022-05-27 02:54:24+00:00,hari rabu lah makin banyak pulak naik ulcer ba...,hari rabu lah makin banyak pulak naik ulcer ba...,-0.3400
...,...,...,...
2022-06-01 23:59:58+00:00,mooners and shakers: bitcoin dives back below ...,mooners shakers bitcoin dives k cardano solana...,0.0772
2022-06-01 23:59:58+00:00,joined a crypto cult today. i’m excited to see...,joined crypto cult today m excited goes,0.3400
2022-06-01 23:59:58+00:00,135/365 📍 hommieee ada late birthday gift nih ...,hommieee ada late birthday gift nih yang dit...,0.4404


In [41]:
ada_data.loc[ada_data.index < '2022-05-27 03:00:00'].sum()

tweet             @dinoxdj99 ada ka jaehyun collector yg drop ? ...
cleaned_tweets    ada ka jaehyun collector yg drop wkwkwkwk nnti...
nltk_polarity                                                8.3303
dtype: object

In [42]:
 8.3303/34

0.24500882352941175

In [43]:
ada_dataset = pd.DataFrame(ada_data.resample('D').mean()['nltk_polarity'])

In [44]:
ada_dataset_daily = pd.DataFrame(ada_data.resample('D').mean()['nltk_polarity'])
btc_dataset_daily = pd.DataFrame(btc_data.resample('D').mean()['nltk_polarity'])
eth_dataset_daily = pd.DataFrame(eth_data.resample('D').mean()['nltk_polarity'])
bin_dataset_daily = pd.DataFrame(bin_data.resample('D').mean()['nltk_polarity'])
xrp_dataset_daily = pd.DataFrame(xrp_data.resample('D').mean()['nltk_polarity'])
sol_dataset_daily = pd.DataFrame(sol_data.resample('D').mean()['nltk_polarity'])
matic_dataset_daily = pd.DataFrame(matic_data.resample('D').mean()['nltk_polarity'])
sand_dataset_daily = pd.DataFrame(sand_data.resample('D').mean()['nltk_polarity'])
algo_dataset_daily = pd.DataFrame(algo_data.resample('D').mean()['nltk_polarity'])
mana_dataset_daily = pd.DataFrame(mana_data.resample('D').mean()['nltk_polarity'])

In [46]:
ada_dataset_hourly = pd.DataFrame(ada_data.resample('H').mean()['nltk_polarity'])
btc_dataset_hourly = pd.DataFrame(btc_data.resample('H').mean()['nltk_polarity'])
eth_dataset_hourly = pd.DataFrame(eth_data.resample('H').mean()['nltk_polarity'])
bin_dataset_hourly = pd.DataFrame(bin_data.resample('H').mean()['nltk_polarity'])
xrp_dataset_hourly = pd.DataFrame(xrp_data.resample('H').mean()['nltk_polarity'])
sol_dataset_hourly = pd.DataFrame(sol_data.resample('H').mean()['nltk_polarity'])
matic_dataset_hourly = pd.DataFrame(matic_data.resample('H').mean()['nltk_polarity'])
sand_dataset_hourly = pd.DataFrame(sand_data.resample('H').mean()['nltk_polarity'])
algo_dataset_hourly = pd.DataFrame(algo_data.resample('H').mean()['nltk_polarity'])
mana_dataset_hourly = pd.DataFrame(mana_data.resample('H').mean()['nltk_polarity'])

In [47]:
ada_dataset_hourly

,nltk_polarity
time,
2022-05-27 02:00:00+00:00,0.245009
2022-05-27 03:00:00+00:00,NaN
2022-05-27 04:00:00+00:00,0.339172
2022-05-27 05:00:00+00:00,0.398100
2022-05-27 06:00:00+00:00,NaN
...,...
2022-06-01 19:00:00+00:00,0.409513
2022-06-01 20:00:00+00:00,NaN
2022-06-01 21:00:00+00:00,0.036009


In [48]:
ada_dataset_daily.to_csv('ada_dataset_daily.csv', encoding='utf-8')

In [49]:
btc_dataset_daily.to_csv('btc_dataset_daily.csv', encoding='utf-8')
eth_dataset_daily.to_csv('eth_dataset_daily.csv', encoding='utf-8')
bin_dataset_daily.to_csv('bin_dataset_daily.csv', encoding='utf-8')
xrp_dataset_daily.to_csv('xrp_dataset_daily.csv', encoding='utf-8')
sol_dataset_daily.to_csv('sol_dataset_daily.csv', encoding='utf-8')
matic_dataset_daily.to_csv('matic_dataset_daily.csv', encoding='utf-8')
sand_dataset_daily.to_csv('sand_dataset_daily.csv', encoding='utf-8')
algo_dataset_daily.to_csv('algo_dataset_daily.csv', encoding='utf-8')
mana_dataset_daily.to_csv('mana_dataset_daily.csv', encoding='utf-8')

In [50]:
btc_dataset_hourly.to_csv('btc_dataset_hourly.csv', encoding='utf-8')
eth_dataset_hourly.to_csv('eth_dataset_hourly.csv', encoding='utf-8')
bin_dataset_hourly.to_csv('bin_dataset_hourly.csv', encoding='utf-8')
xrp_dataset_hourly.to_csv('xrp_dataset_hourly.csv', encoding='utf-8')
sol_dataset_hourly.to_csv('sol_dataset_hourly.csv', encoding='utf-8')
matic_dataset_hourly.to_csv('matic_dataset_hourly.csv', encoding='utf-8')
sand_dataset_hourly.to_csv('sand_dataset_hourly.csv', encoding='utf-8')
algo_dataset_hourly.to_csv('algo_dataset_hourly.csv', encoding='utf-8')
mana_dataset_hourly.to_csv('mana_dataset_hourly.csv', encoding='utf-8')